# Рекомендация тарифов

## Откройте и изучите файл

**Библиотеки**

In [6]:
import pandas as pd
import math
import sklearn
import numpy as np
import random

Функции из библиотеки sklearn

In [7]:
# Импортируем функцию из бибилиотеки sklearn
from sklearn.model_selection import train_test_split

# Импортируем метод дерева принятия решений
from sklearn.tree import DecisionTreeClassifier

# Импортируем метод оценки доли правильных ответов
from sklearn.metrics import accuracy_score

# Импортируем алгоритм случайного леса
from sklearn.ensemble import RandomForestClassifier

# Импортируем алгоритм случайного поиска по гиперпараметрам
from sklearn.model_selection import RandomizedSearchCV

# Импортируем метод логистической регрессии
from sklearn.linear_model import LogisticRegression

**Открываем файл**

In [8]:
df = pd.read_csv('/Users/artemvishanov/Desktop/yandex_practicum_project/6. Рекомендация тарифов/users_behavior.csv')

**Рассмотрим датафрейм:**

In [9]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропуски отсутствуют.

**Вывод:**

* Данные не нуждаются в предобработке.
* Пропуски отсутствуют.

## Разбейте данные на выборки

Разобьем данные на три выборки: обучающую, валидационную и тестовую. Разобьем их в отношении 3:1:1. Пока нами не была изучена кросс-валидация, использую для этого метод train_test_split():

In [11]:
# Поделим датафрейм на обучающую выборку и выборку, которую позже разделим на валидационную и тестовую.
df_train, df_divide = train_test_split(df, test_size=0.4, random_state=12345)

# Поделим df_divide валидационную и тестовую выборку.
df_valid, df_test = train_test_split(df_divide, test_size=0.5, random_state=12345)

**Вывод:** Выборки разделены.

## Исследуйте модели

**Перед исследованием моделей машинного обучения разделим наши датафреймы на целевой признак который нужно предсказать и признаки, которые помогут нам его предсказать:**

In [12]:
# Разделим обучающий датафрейм на features и target - целевой признак
features_train = df_train.drop(['is_ultra'], axis = 1)
target_train = df_train['is_ultra']

# Разделим валидационный датафрейм на features и target - целевой признак
features_valid = df_valid.drop(['is_ultra'], axis = 1)
target_valid = df_valid['is_ultra']

**Оценим в цикле долю правильных ответов для разных глубин дерева принятия решений:**

In [13]:
for depth in range(2,27,2):
    model = DecisionTreeClassifier(random_state=0, max_depth=depth, min_samples_split= 100)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('max_depth =',depth,': {:.4f}'.format(accuracy))

max_depth = 2 : 0.7823
max_depth = 4 : 0.7854
max_depth = 6 : 0.7885
max_depth = 8 : 0.7854
max_depth = 10 : 0.7807
max_depth = 12 : 0.7807
max_depth = 14 : 0.7807
max_depth = 16 : 0.7807
max_depth = 18 : 0.7807
max_depth = 20 : 0.7807
max_depth = 22 : 0.7807
max_depth = 24 : 0.7807
max_depth = 26 : 0.7807



Лучшая модель с глубиной 6 с accuracy = 0.7885. min_samples_split был подобран интуитивно, и улучшил результат на 6 тысячных по сравнению с дефолтным. Любые опыты с критерием и весом классов (balanced) ухудшали результат при min_samples_split = 100.

**Перейдем к алгоритму случайного леса:**

Оценим в цикле долю правильных ответов для разного количества оценщиков

In [14]:
for estimators in range(2,16):
    model = RandomForestClassifier(random_state=8897, n_estimators=estimators, max_depth=6)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('n_estimators =',estimators,': {:.4f}'.format(accuracy))

n_estimators = 2 : 0.7776
n_estimators = 3 : 0.7900
n_estimators = 4 : 0.7916
n_estimators = 5 : 0.7947
n_estimators = 6 : 0.7947
n_estimators = 7 : 0.7916
n_estimators = 8 : 0.7900
n_estimators = 9 : 0.7947
n_estimators = 10 : 0.8009
n_estimators = 11 : 0.7994
n_estimators = 12 : 0.8056
n_estimators = 13 : 0.8040
n_estimators = 14 : 0.8009
n_estimators = 15 : 0.8025


Лучшая модель с количеством оценщиков 12 и глубиной 6 с accuracy = 0.8056. min_samples_split лучше оставить дефолтным, он ухудшал результат.

In [15]:
# сетка гиперпараметров, по которой будет происходит случайный поиск
param_grid = {
    'n_estimators': np.linspace(2, 100).astype(int),
    'max_depth': [None] + list(np.linspace(2, 10).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0, 5, 1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# случайный лес к которому будем подбирать параметры
estimator = RandomForestClassifier(random_state = 8897)

# модель
rs = RandomizedSearchCV(estimator, param_grid, random_state=8897)

# обучаем модель
rs.fit(features_train, target_train)

RandomizedSearchCV(estimator=RandomForestClassifier(random_state=8897),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 2, 2, 2, 2, 2, 2, 2,
                                                      3, 3, 3, 3, 3, 3, 4, 4, 4,
                                                      4, 4, 4, 5, 5, 5, 5, 5, 5,
                                                      6, 6, 6, 6, ...],
                                        'max_features': ['auto', 'sqrt', None,
                                                         0, 1, 2, 3, 4],
                                        'max_leaf_nodes': [None, 10, 10, 10, 10,
                                                           10, 10, 10, 10, 10,
                                                           10, 10, 10, 10, 11,
                                                           11, 11, 11, 11, 11,
                                                           11, 11, 11, 11, 

In [16]:
# набор параметров, которые подобрал случайный поиск.
rs.best_params_

{'n_estimators': 20,
 'min_samples_split': 10,
 'max_leaf_nodes': 37,
 'max_features': 4,
 'max_depth': 8,
 'bootstrap': True}

In [17]:
# предскажем целевой признак и подсчитаем долю правильных ответов.
predictions = rs.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
accuracy

0.8009331259720062

**Перейдем к логистической регрессии:**

In [18]:
# Создадим модель лог. регрессии
model = LogisticRegression(random_state=12345)

# обучим ее
model.fit(features_train, target_train)

# предскажем целевой признак
predictions = model.predict(features_valid)

# и подсчитаем долю правильных ответов.
accuracy = accuracy_score(target_valid, predictions)

# выведем долю правильных ответов
accuracy

0.7107309486780715

**Вывод:**

* Мы разделили тренировочную выборку и валидационную на признаки(features) и целевой признак(target).
* Обучили дерево принятия решений и выбрали лучшие с максимальной глубиной 2 и 6.
* Обучили модель случайного леса, и выбрали лучшую с количеством оценщиков 12 и глубиной 6. Ее используем для проверки на тестовой выборке. Также используем модель, для которой подобрали параметры случайным поиском.
* Обучили модель логистической регресии и получили метрики хуже чем у предыдущих моделей.
* Открыли для себя несколько интересных методов, гиперпараметров, потренировались подбирать параметры.

## Проверьте модель на тестовой выборке

**Перед проверкой модели разделим наш тестовый датафрейм на features и target:**

In [19]:
# Разделим тестовый датафрейм на features и target - целевой признак
features_test = df_test.drop(['is_ultra'], axis=1)  
target_test = df_test['is_ultra']

**Проверим модель на тестовой выборке:**

In [20]:
# Создадим модель случайного леса
model = RandomForestClassifier(random_state=8897, n_estimators=12, max_depth=6)

# обучим ее
model.fit(features_train, target_train)

# предскажем целевой признак
predictions = model.predict(features_test)

# и подсчитаем долю правильных ответов
accuracy = accuracy_score(target_test, predictions)

# выведем долю правильных ответов
print('accuracy =','{:.4f}'.format(accuracy))

accuracy = 0.7994


**Проверим модель rs с гиперпараметрами, подобранными случайным поиском на тестовой выборке:**

In [21]:
# предскажем целевой признак
predictions = rs.predict(features_test)

# и подсчитаем долю правильных ответов
accuracy = accuracy_score(target_test, predictions)

# выведем долю правильных ответов
print('accuracy =','{:.4f}'.format(accuracy))

accuracy = 0.8087


Модель, для которой мы подбирали гиперпараметры случайным поиском показала чуть-чуть лучший результат, чем та у которой мы подобрали только глубину и количество оценщиков. Тем не менее обе модели справились с поставленной задачей.

**Вывод:**

* Мы проверили две лучшие подобранные нами модели на тестовой выборке и получили устраивающий нас результат.
* Следующий шаг: научиться пользоваться кросс-валидацией и GridSearch для повышения accuracy в подобных задачах.

## (бонус) Проверьте модели на адекватность

**Для проверки адекватности нашей модели напишем модель (функцию), предсказывающую значение признака is_ultra самым простым и тупорылым способом, 50 на 50, и сравним, лучше ли наша модель:**

In [22]:
random_predictions = np.random.randint(low = 0, high = 2, size = 643) 

# подсчитаем долю правильных ответов
accuracy = accuracy_score(target_test, random_predictions)

# выведем долю правильных ответов
print('accuracy =','{:.4f}'.format(accuracy))

accuracy = 0.5054


Как видим случайные ответы ошибаются примерно в 50% случаев. Наша модель ошибается реже - только лишь в 19.44% случаев.

**Вывод:**
* Наша модель лучше, чем подкидывание монетки. Она прошла проверку на адекватность/вменяемость.